In [59]:
import pymongo, sys,os
client = pymongo.MongoClient('localhost',27017)
slide_db_ptr = client['TCGA_Slide_DB']
pyr_db_ptr = client['CDSA_Pyramid_DB']


pyramid_root = '/PYRAMIDS/PYRAMIDS/CDSA/' 
tcga_mirror_root = '/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/' 

In [6]:
### Get the Slide Collection Names

slide_collections = slide_db_ptr.collection_names(False)

In [80]:
#  vips_command = 'vips im_vips2tiff -v '+output_file+' '+pyramid_file+':jpeg:90,tile:256x256,pyramid,,,,8 '
# dsa_mongo_common_functions.py:          vips_command = 'vips im_vips2tiff -v '+output_file+' '+pyramid_file+':jpeg:90,tile:256x256,pyramid,,,,8 '
# dsa_mongo_common_functions.py:          print vips_command
# dsa_mongo_common_functions.py:          current_command_list += vips_command
# regenerate_black_cdsa_images.py:        vips_command = 'vips im_extract_bands --vips-progress -v '+svs_file+' '+pyramid_file+':jpeg:75,tile:256x256,pyramid,,,,8 0 3'
# regenerate_black_cdsa_images.py:        print vips_command
# regenerate_black_cdsa_images.py:        #current_command_list += vips_command
# simple_svs_and_pyramid_latest_archive_scanner.py:vips_fp = open('LOG_FILES/vips_job_file.txt','w');
# simple_svs_and_pyramid_latest_archive_scanner.py:       #vips_fp.write( "vips im_extract_bands "+svs_path+pyramid_path+":jpeg:75,tile:256x256,pyramid,,,,8 0 3\r\n" )
# simple_svs_and_pyramid_latest_archive_scanner.py:                               vips_fp.write( "vips im_extract_bands "+svs_path+' '+pyramid_path+":jpeg:75,tile:256x256,pyramid,,,,8 0 3\r\n" )
# simple_svs_and_pyramid_latest_archive_scanner.py:               vips_fp.write( "vips im_extract_bands "+svs_path+' '+pyramid_path+":jpeg:75,tile:256x256,pyramid,,,,8 0 3\r\n" )
# svs_and_pyramid_latest_archive_scanner.py:vips_fp = open('LOG_FILES/vips_job_file.txt','w');
# svs_and_pyramid_latest_archive_scanner.py:      #vips_fp.write( "vips im_extract_bands "+svs_path+pyramid_path+":jpeg:75,tile:256x256,pyramid,,,,8 0 3\r\n" )

vips_fp = open('LOG_FILES/vips_job_file.txt','w')

for sc in slide_collections:
    #print "ANALYZING", sc,"Tumor Type"
    ### First get the slide collection data, this is at the tumor type level
    ## The pyramid collections are organized a bit differently, so I need to map the collection names between the systems
    svs_files = slide_db_ptr[sc].find()
    svs_w_pyramid = 0
    needs_pyramid = 0
    for svs in svs_files:
        slide_name = svs['slide_name']
        prep_type = svs['prep_type']
        #print svs
        pyramid_collection_name =  map_archive_name(sc, prep_type)
        
        pyramid_name = slide_name+'.dzi.tif'
        #print "Looking for",pyramid_name,pyramid_collection_name
        pyr_query =  pyr_db_ptr[pyramid_collection_name].find_one({'pyramid_name': pyramid_name})
        if pyr_query:
            #print pyr_query, slide_name, prep_type
            svs_w_pyramid +=1
        else:
            needs_pyramid +=1
            slide_w_path = svs['slide_w_path']
            prep_type = svs['prep_type']
            ### doing a set of nonintuitive transformatinos for the pyramid output directory
            
            archive_dir = slide_w_path.split('/')[-2]
            ## #Need to make sure output directory is created or the conversion failes
            output_pyr_dir = os.path.join(pyramid_root, map_archive_name(sc,prep_type),archive_dir )
            if not os.path.isdir(output_pyr_dir):
                print "Need to make output dir!",output_pyr_dir
                
                #os.makedirs(output_pyr_dir)
                vips_fp.write( "mkdir -p "+output_pyr_dir+"\r\n" )
                ### This code may need some thought... I am running this process as dagutman or root... may
                ### Screw up the file system
            
            pyr_w_path  = os.path.join( output_pyr_dir, pyramid_name)
            #print output_pyr_dir, pyr_w_path
                        
                        
            vips_fp.write( "vips im_extract_bands "+slide_w_path+pyr_w_path+":jpeg:75,tile:256x256,pyramid,,,,8 0 3\r\n" )

            
            #sys.exit()
            
            #print pyramid_name  slide_w_path
            

    print sc,"needs",needs_pyramid,"and these svs files seem to have pyramids",svs_w_pyramid
    #sys.exit()
    # regenerate_black_cdsa_images.py:# 


prad needs 338 and these svs files seem to have pyramids 940
acc needs 6 and these svs files seem to have pyramids 323
Need to make output dir! /PYRAMIDS/PYRAMIDS/CDSA/GBM_Diagnostic/intgen.org_GBM.diagnostic_images.Level_1.10.1.0
Need to make output dir! /PYRAMIDS/PYRAMIDS/CDSA/GBM_Diagnostic/intgen.org_GBM.diagnostic_images.Level_1.10.1.0
Need to make output dir! /PYRAMIDS/PYRAMIDS/CDSA/GBM_Diagnostic/intgen.org_GBM.diagnostic_images.Level_1.10.1.0
Need to make output dir! /PYRAMIDS/PYRAMIDS/CDSA/GBM_Diagnostic/intgen.org_GBM.diagnostic_images.Level_1.10.1.0
Need to make output dir! /PYRAMIDS/PYRAMIDS/CDSA/GBM_Frozen/intgen.org_GBM.tissue_images.Level_1.130.2.0
gbm needs 28 and these svs files seem to have pyramids 2475
Need to make output dir! /PYRAMIDS/PYRAMIDS/CDSA/LGG_Diagnostic/intgen.org_LGG.diagnostic_images.Level_1.163.0.0
Need to make output dir! /PYRAMIDS/PYRAMIDS/CDSA/LGG_Diagnostic/intgen.org_LGG.diagnostic_images.Level_1.163.0.0
Need to make output dir! /PYRAMIDS/PYRAMID

In [53]:
def map_archive_name( tissue_type, prep_type):
    """ This helper function helps with an idiosyncratic issue where the tcga organizes data at the top level by 
    simply the cancer type, but I decided to organize by cancer type AND prep type (e.g. diagnostic or tissue)"""
    #print tissue_type,"is the tissue type"
    if prep_type == 'tissue_image':
        prep_type = 'Frozen'
    elif prep_type == 'diagnostic':
        prep_type = 'Diagnostic'
    
    pyramid_collection_name = tissue_type.upper()+'_'+prep_type
    return pyramid_collection_name